<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/testing_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
!unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [ ]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

## Processing data

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [ ]:
pathfiles = list()
for pathdir in glob.glob('/content/ViMs/original/*'):
  for pathfile in glob.glob(pathdir + '/original/*'):
    pathfiles.append(pathfile)

In [ ]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ''
    summary = ''
    start_copy_summary = False
    start_copy_content= False

    for row in rows:
      if row[:7] == 'Content':
        start_copy_summary = False
        start_copy_content = True
      elif start_copy_content:
        original += row + ' '
      elif start_copy_summary:
        summary += row + ' '
      elif row[:7] == 'Summary': 
        start_copy_summary = True
        summary += row[9:] + ' '

    original = rdrsegmenter.tokenize(original)
    original = ' '.join([' '.join(x) for x in original])

    summary = rdrsegmenter.tokenize(summary)
    summary = ' '.join([' '.join(x) for x in summary])

    if summary == '':
      summary = None
    if original == '':
      original = None
      
    return {'file' : pathfile,
            'original': original, 
            'summary': summary}

In [ ]:
read_content('/content/ViMs/original/Cluster_001/original/8.txt')

{'file': '/content/ViMs/original/Cluster_001/original/8.txt',
 'original': 'Bộ_trưởng Quốc_phòng Hy_Lạp Panos_Kammenos , máy_bay đã " quay 90 độ sang trái và 360 độ sang phải , hạ độ cao từ 37.000 feet xuống 15.000 feet và mất tín_hiệu ở độ cao 10.000 feet " ( khoảng 11 km ) . Theo lời quan_chức này , chiếc máy_bay MS804 bất_ngờ bay chậm lại ở độ cao khoảng hơn 11km . Ở toạ_độ này , máy_bay " bất_ngờ chuyển_hướng " , ban_đầu xoay 90 độ sang bên phải , sau đó lại xoay ngược về bên trước khi biến mất khỏi màn_hình radar và có_thể là bắt_đầu rơi . Theo nguồn tin sân_bay Hy_Lạp cho biết , chiếc máy_bay MS804 đã rơi ở ngoài khơi đảo Karpathos của nước này . Truyền_thông Hy_Lạp cũng đưa tin rằng , ngư_dân trên một tàu đánh_cá của Hy_Lạp đã nhìn thấy một vệt sáng trên bầu_trời ở Địa_Trung_Hải . Tuy_nhiên , thông_tin này hiện chưa được kiểm_chứng . Một đoạn video xuất_hiện trên mạng Twitter được cho là quay lại khoảnh_khắc máy_bay của hãng hàng_không EgyptAir bốc cháy như một quả cầu lửa trên 

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
  data = executor.map(read_content, pathfiles)

In [ ]:
# Make blank dataframe
data_df = list()
for d in data:
  data_df.append(d)
data_df = pd.DataFrame(data_df)
data_df

,file,original,summary
0,/content/ViMs/original/Cluster_298/original/19...,"Thông_tin trên Pravda cho biết , sự_việc hy_hữ...",Một chiếc máy_bay chở 147 hành_khách vừa đáp x...
1,/content/ViMs/original/Cluster_298/original/19...,"Hãng tin Pravda cho biết , sự_việc xảy ra_chiề...",Đường_băng ở sân_bay quốc_tế Domodedovo ở Mosc...
2,/content/ViMs/original/Cluster_298/original/19...,"Báo Pravda của Nga đưa tin , hôm 2/4 , đường_b...",Một sự_cố hi_hữu đã xảy ra khi một chiếc máy_b...
3,/content/ViMs/original/Cluster_298/original/19...,Chiếc máy bay đã đứng tại chỗ trong khoả...,Một chiếc máy_bay thực hiện chuyến bay từ ...
4,/content/ViMs/original/Cluster_298/original/19...,Theo những tin_tức mới nhất báo Dân_Trí trích ...,"ai nạn hy_hữu xảy ra ở Nga , khi đường_băng ở ..."
...,...,...,...
1940,/content/ViMs/original/Cluster_157/original/10...,"Sáng nay , khi cơ_quan điều_tra vào_cuộc điều_...","Trưa nay ( 29-5 ) , nguồn tin cho biết trong v..."
1941,/content/ViMs/original/Cluster_157/original/10...,"Trước đó , báo Người đưa tin đã thông_tin , kh...",Trong khi bị lực_lượng chức_năng tổ_chức vây b...
1942,/content/ViMs/original/Cluster_157/original/10...,"Sáng nay , Công_an huyện Quế_Phong ( Nghệ_An )...","Tỉnh_giấc sau chầu nhậu , Thắng ra_tay giết ch..."
1943,/content/ViMs/original/Cluster_157/original/10...,"Theo đó , khi cơ_quan công_an huyện Quế_Phong ...","Sáng 29/5 , ông Lương_Trường_Sơn - Trưởng CA x..."


In [ ]:
data_df.dropna(inplace = True)
data_df

,file,original,summary
0,/content/ViMs/original/Cluster_298/original/19...,"Thông_tin trên Pravda cho biết , sự_việc hy_hữ...",Một chiếc máy_bay chở 147 hành_khách vừa đáp x...
1,/content/ViMs/original/Cluster_298/original/19...,"Hãng tin Pravda cho biết , sự_việc xảy ra_chiề...",Đường_băng ở sân_bay quốc_tế Domodedovo ở Mosc...
2,/content/ViMs/original/Cluster_298/original/19...,"Báo Pravda của Nga đưa tin , hôm 2/4 , đường_b...",Một sự_cố hi_hữu đã xảy ra khi một chiếc máy_b...
3,/content/ViMs/original/Cluster_298/original/19...,Chiếc máy bay đã đứng tại chỗ trong khoả...,Một chiếc máy_bay thực hiện chuyến bay từ ...
4,/content/ViMs/original/Cluster_298/original/19...,Theo những tin_tức mới nhất báo Dân_Trí trích ...,"ai nạn hy_hữu xảy ra ở Nga , khi đường_băng ở ..."
...,...,...,...
1940,/content/ViMs/original/Cluster_157/original/10...,"Sáng nay , khi cơ_quan điều_tra vào_cuộc điều_...","Trưa nay ( 29-5 ) , nguồn tin cho biết trong v..."
1941,/content/ViMs/original/Cluster_157/original/10...,"Trước đó , báo Người đưa tin đã thông_tin , kh...",Trong khi bị lực_lượng chức_năng tổ_chức vây b...
1942,/content/ViMs/original/Cluster_157/original/10...,"Sáng nay , Công_an huyện Quế_Phong ( Nghệ_An )...","Tỉnh_giấc sau chầu nhậu , Thắng ra_tay giết ch..."
1943,/content/ViMs/original/Cluster_157/original/10...,"Theo đó , khi cơ_quan công_an huyện Quế_Phong ...","Sáng 29/5 , ông Lương_Trường_Sơn - Trưởng CA x..."


In [ ]:
data_df = data_df.sample(frac=1).reset_index(drop=True)
data_df

,file,original,summary
0,/content/ViMs/original/Cluster_146/original/94...,Vụ_việc trên xảy ra vào_khoảng 15h30 chiều nay...,Sau nhiều lần đòi gặp người đứng đầu tập_đoàn ...
1,/content/ViMs/original/Cluster_297/original/19...,"Chiều 3/6 , trao_đổi với PV Báo Gia_đình & Xã_...","Sau khi hiếp_dâm nữ_sinh lớp 11 , tên yêu râu ..."
2,/content/ViMs/original/Cluster_271/original/17...,Dự_án sẽ được thi_công hai giai_đoạn . Tổng mứ...,Công_trình nhằm xoá điểm kẹt xe vào cảng Cát_L...
3,/content/ViMs/original/Cluster_007/original/56...,Anh Phan_Đức_Huy đến bãi thu_mua lốp xe cũ ở đ...,Trong lúc kéo điện ra hàn ôtô tại bãi thu_mua ...
4,/content/ViMs/original/Cluster_051/original/33...,Vụ_việc ông Nguyễn_Phi_Cường ( quản_lý của ca_...,Hai người quen dàn_cảnh lừa mua nhà gần 4 tỉ đ...
...,...,...,...
1895,/content/ViMs/original/Cluster_263/original/17...,Cuộc họp_báo diễn ra ngày 1-6 ( giờ_địa_phương...,""" Cô biết gì về Trung_Quốc ? Cô đi Trung_Quốc ..."
1896,/content/ViMs/original/Cluster_245/original/16...,"Chiều 2/6 , Chính_phủ họp_báo thường_kỳ tháng ...",“ Đến nay các nhà_khoa_học đã xác_định được ng...
1897,/content/ViMs/original/Cluster_294/original/19...,Vụ_việc xảy ra khoảng 4h sáng nay ( 3.6 ) tại ...,"Do nghi_ngờ dẫn đến ghen_tuông , đối_tượng đã ..."
1898,/content/ViMs/original/Cluster_081/original/53...,"Ông Alexander_Van der Bellen ( 72 tuổi , chuyê...",Lãnh_đạo châu_Âu cảm_thấy nhẹ_nhõm do Tây_Âu s...


## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [ ]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [ ]:
from transformers import RobertaTokenizerFast,AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# train_data = datasets.load_dataset("xsum", split="train")
# val_data = datasets.load_dataset("xsum", split="validation[:10%]")

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data_df, test_size=0.2)
train_data =  Dataset.from_pandas(train_data)
val_data =  Dataset.from_pandas(val_data)

In [ ]:
batch_size=4  # change to 16 for full training
encoder_max_length=256
decoder_max_length=64

def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

### **Warm-starting the Encoder-Decoder Model**

In [ ]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base", tie_encoder_decoder=True)

In [ ]:
roberta_shared1 = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)

In [ ]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 64
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size  

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [ ]:
%%capture
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [ ]:
import datasets

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Cool! Finally, we start training.

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=2000,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=7500,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    # max_steps=16, # delete for full training
    num_train_epochs=5, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..
/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the le

Step,Training Loss


TrainOutput(global_step=1900, training_loss=5.392902446546053, metrics={'train_runtime': 574.316, 'train_samples_per_second': 3.308, 'total_flos': 2393398065408000, 'epoch': 5.0})

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained("./checkpoint-1500")
model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

# # only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# test_data = test_data.select(range(16))
train_data, val_data = train_test_split(data_df, test_size=0.2)
train_data =  Dataset.from_pandas(train_data)
val_data =  Dataset.from_pandas(val_data)

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = val_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]



Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

NameError: ignored

In [ ]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

Score(precision=0.1505006404700973, recall=0.12705074895880886, fmeasure=0.1319454793696977)


In [ ]:
pred_str

['Ngày 1/6, tại TP. HCM đã tổ_chức Đại_hội cổ_đông thường_niên năm 2016 với tên gọi Đại_hội cổ_đông năm 2015, với tên là Đại_hội_đồng cổ_đông năm 2016.',
 'Trong đó, nữ diễn_viên xinh_đẹp nhất Hàn_Quốc đã đảm_nhận vai_trò diễn_viên chính trong bộ phim tình_cảm lãng_mạn, lãng_mạn.',
 'Trong đó, người_dân phát_hiện thi_thể một nam thanh_niên đang nằm bên vệ đường, nghi bị sát_hại.',
 'Theo thông_tin ban_đầu được đăng_tải trên mạng xã_hội Thổ_Nhĩ_Kỳ cho biết, hãng hàng_không EgyptAir đã hạ_cánh xuống biển Địa_Trung_Hải sau khi hạ_cánh xuống sân_bay quốc_tế Istanbul.',
 'Sáng nay ( 1/6 ), tại Hà_Nội, Bộ NN PTNT PTNT & & PTNT ) cho biết, đơn_vị này đang phối_hợp với các địa_phương, các tỉnh, thành, thành_phố, thành_phố phía phía Nam, vùng miền Trung, các các tỉnh phía Trung, Tây_Nguyên, miền Trung và miền Trung - - Trung - Trung,',
 'Trong trận đấu giữa hai đội đội bóng của Anh, tiền_vệ ĐT ĐT Anh vừa được HLV Roy_Hodgson triệu_tập vào danh_sách 23 cầu_thủ tham_dự Euro 2016.',
 'Theo thông_t

In [ ]:
label_str

['“ Bộ Tài_chính đề_nghị Ngân_hàng Nhà_nước chỉ_đạo người đại_diện phần vốn Nhà_nước tại Ngân_hàng TMCP Đầu_tư và Phát_triển ( BIDV ) và Ngân_hàng TMCP Công_thương ( Vietinbank ) biểu_quyết chia cổ_tức năm tài_chính 2015 bằng tiền_mặt và nộp toàn_bộ số cổ_tức được chia cho phần vốn Nhà_nước tại hai ngân_hàng trên vào ngân_sách nhà_nước ” – đây là nội_dung chính của văn_bản số 6854 / BTC-TCT ngày 19-5-2016 của Bộ Tài_chính gửi Thống_đốc NHNN .',
 'Mẹ Rồng của Game of Thrones đã tâm_sự rằng mình phải uống rượu để say mới dám đóng các cảnh nóng .',
 'Với sự hiếu_kì và tò_mò , hàng trăm người_dân đã tập_trung về gia_đình anh Nguyễn_Hữu_Thông , tại khối 13 , thị_trấn Phố Châu , huyện Hương_Sơn ( Hà_Tĩnh ) để xem thần trăn xuất_hiện tại đây .',
 'Bức ảnh từ hai năm trước của nữ tiếp_viên trên chuyến bay MS804 của EgyptAir chụp ghép với một máy_bay chìm xuống biển đang gây xôn_xao dư_luận giữa lúc vụ mất_tích MS804 vẫn là một bí_ẩn lớn .',
 'Trước hiện_tượng cá chết hàng_loạt ở miền Trung , N

The fully trained *RoBERTaShared* model is uploaded to the 🤗model hub under [patrickvonplaten/roberta_shared_bbc_xsum](https://huggingface.co/patrickvonplaten/roberta_shared_bbc_xsum). 

The model achieves a ROUGE-2 score of **16.89**, which is a bit worse than reported in the paper. Training the model for a bit longer will most likely close this performance gap.

For some summarization examples, the reader is advised to use the online inference API of the model, [here](https://huggingface.co/patrickvonplaten/roberta_shared_bbc_xsum).